In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [2]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixregV2.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixregV2.csv
day8_features_matrixregV2.csv
day9_features_matrixregV2.csv
test_features_matrixregV2.csv


# Load Ip

In [3]:
# path = '/home/kai/data/kaggle/talkingdata/data/'
# train = pd.read_csv(path + 'train_cleaned_final.csv')
# print('loading training done')
# test = pd.read_csv(path + 'test_cleaned_final.csv')
# print('loading testing done')

# df_hour = pd.read_csv(path+'hourdistri.csv', index_col='Unnamed: 0')
# index = {}
# for day in ['day7', 'day8','day9']:
#     print(day)
#     index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
#     list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
#     list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 
#     df_dict[day]['ip'] = train.loc[index[day], 'ip'].values
# print('processing test!')
# df_dict['test']['ip'] = test['ip']


# just use categorical values that exist in both train and test
no improvement  
ip will not help

In [4]:
# def intersec_category(df_train_all, df_test_all, df, col_list):
#     for col in col_list:
#         print('processing: {}'.format(col))
#         train_index = set(df_train_all[col].value_counts().index)
#         test_index = set(df_test_all[col].value_counts().index)                 
#         inter_index = list(train_index.intersection(test_index))
#         print('train index length is: {}'.format(len(train_index)))
#         print('test index length is: {}'.format(len(test_index)))
#         print('intersection index length is: {}'.format(len(inter_index)))
#         index_map = pd.Series(inter_index, index=inter_index)
#         df.loc[:,col] = df[col].map(index_map).fillna(-1)
#     return df

# col_list = [ 'app', 'device', 'os', 'channel']
# df_train_all = pd.concat([df_dict[day_list[0]],df_dict[day_list[1]], df_dict[day_list[2]]])
# # for file in ['day7', 'day8', 'day9','test']: 
# for file in ['day7']:
#     temp = intersec_category(df_train_all, df_dict['test'], df_dict[file],col_list)

# define parameter

In [56]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':50,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
# categorical_col = [ 'app', 'device', 'os', 'hour']
feature_cols = list(df_dict['test'].columns.values)

In [57]:
df_dict['day7'].columns.values

array(['ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean',
       'ip_app_device_os_mean', 'ip_app_device_mean', 'app_device_os_mean',
       'ip_device_os_time2nextclick', 'ip_app_device_os_time2nextclick',
       'ip_app_device_time2nextclick', 'app_device_os_time2nextclick',
       'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_app_device_time2previousclick',
       'app_device_os_time2previousclick', 'ip_device_os_countfromfuture',
       'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture',
       'app_device_os_countfromfuture', 'ip_device_os_countfrompast',
       'ip_app_device_os_countfrompast', 'ip_app_device_countfrompast',
       'app_device_os_countfrompast', 'ip_device_os_lasttimediff',
       'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff',
  

In [58]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
extra = set(['app_device_os_mean',
             'app_device_os_time2nextclick', 
             'app_device_os_time2previousclick',
             'app_device_os_countfromfuture',
             'app_device_os_countfrompast', 
             'app_device_os_lasttimediff', 
             'app_device_os_regression' ])
# extra = set(['ip_app_device_mean',
#              'ip_app_device_time2nextclick', 
#              'ip_app_device_time2previousclick',
#              'ip_app_device_countfromfuture',
#              'ip_app_device_countfrompast', 
#              'ip_app_device_lasttimediff', 
#              'ip_app_device_regression'])
feature_cols = list(set(feature_cols) - extra)

In [59]:
feature_cols

['ip_app_device_os_time2nextclick',
 'hour',
 'ip_os_day_hour_count',
 'ip_app_device_os_countfrompast',
 'ip_device_os_countfrompast',
 'ip_app_device_mean',
 'ip_app_device_os_mean',
 'ip_app_os_day_hour_count',
 'ip_app_device_countfromfuture',
 'device',
 'ip_device_os_time2nextclick',
 'ip_device_os_mean',
 'ip_device_os_countfromfuture',
 'app',
 'ip_app_device_lasttimediff',
 'ip_device_os_time2previousclick',
 'ip_app_device_os_regression',
 'ip_app_device_os_count',
 'ip_app_day_hour_count',
 'ip_device_os_count',
 'ip_device_os_regression',
 'ip_app_device_regression',
 'matrixFact_user_ip_item_appdeviceos',
 'ip_app_device_os_time2previousclick',
 'ip_app_device_os_lasttimediff',
 'channel',
 'os',
 'ip_app_device_time2previousclick',
 'ip_app_device_countfrompast',
 'ip_device_os_lasttimediff',
 'matrixFact_user_iposdeviceapp_item_app',
 'ip_app_device_time2nextclick',
 'ip_app_device_os_countfromfuture']

In [60]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [61]:
for day in day_list:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_predtest_matrixregV2_exclude_leaf61_no2counts.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
    df_all = pd.concat([trainset, valset])
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()
    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)
    
    # prediction
    df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_{}{}_val_{}_matrixregV2_exclude_leaf61_no2counts.csv.gz'.format(train_day[0],train_day[1],day), compression='gzip', index=False)

    

concat_day8day9_val_day7_predtest_matrixregV2_exclude_leaf61_no2counts.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.980302
[20]	valid_0's auc: 0.981693
[30]	valid_0's auc: 0.982578
[40]	valid_0's auc: 0.983448
[50]	valid_0's auc: 0.983952
[60]	valid_0's auc: 0.98422
[70]	valid_0's auc: 0.984284
[80]	valid_0's auc: 0.984378
[90]	valid_0's auc: 0.984422
[100]	valid_0's auc: 0.984403
[110]	valid_0's auc: 0.984397
[120]	valid_0's auc: 0.984434
[130]	valid_0's auc: 0.984429
[140]	valid_0's auc: 0.984434
[150]	valid_0's auc: 0.984416
[160]	valid_0's auc: 0.984405
[170]	valid_0's auc: 0.984414
Early stopping, best iteration is:
[122]	valid_0's auc: 0.984446
start training
[10]	valid_0's auc: 0.981465
[20]	valid_0's auc: 0.983073
[30]	valid_0's auc: 0.984396
[40]	valid_0's auc: 0.985695
[50]	valid_0's auc: 0.986497
[60]	valid_0's auc: 0.987183
[70]	valid_0's auc: 0.987738
[80]	valid_0's auc: 0.98821
[90]	valid_0's auc: 0.988666
[100]	valid_0's auc: 0.989039
[110]	valid_0's auc: 0.989447
[120]	valid_0's auc: 0.989801
loading 

In [46]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

app                                       873
os                                        858
ip_app_device_os_time2nextclick           592
matrixFact_user_ip_item_appdeviceos       436
matrixFact_user_iposdeviceapp_item_app    369
ip_day_hour_count                         363
ip_device_os_count                        353
ip_app_device_os_lasttimediff             231
ip_app_device_os_count                    228
ip_app_device_time2nextclick              194
ip_app_device_lasttimediff                191
ip_device_os_lasttimediff                 190
app_day_hour_count                        186
ip_app_device_countfromfuture             169
ip_app_device_countfrompast               167
hour                                      163
ip_app_device_time2previousclick          156
ip_os_day_hour_count                      155
ip_device_os_countfromfuture              141
ip_device_os_countfrompast                136
ip_app_device_mean                        133
ip_device_os_mean                 

# Average

In [63]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_predtest_matrixregV2_exclude_leaf61_no2counts.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_predtest_matrixregV2_exclude_leaf61_no2counts.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_predtest_matrixregV2_exclude_leaf61_no2counts.npy')

pred = (day7 + day8 + day9)/3

In [64]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_blend_matrixregV2_exclude_leaf61_no2counts.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
